In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import folium

In [2]:
home_class = pd.read_csv('data/train/S1/D1S1_Home.csv')
garage_class = pd.read_csv('data/train/S1/D1S1_ParkingGarage.csv')
roam_class = pd.read_csv('data/train/S1/D1S1_Roam.csv')

In [3]:
print(home_class.shape, garage_class.shape, roam_class.shape)

(547, 40) (628, 40) (216, 40)


In [4]:
def plot_movement(data):
    fig1 = go.Figure()
    for ax in ['accelerometer_x', 'accelerometer_y', 'accelerometer_z']:
        fig1.add_trace(go.Scatter(x=data['seconds_elapsed'], y=data[ax], name=ax))
    fig1.show()
    fig2 = go.Figure()
    for ax in ['gyroscope_x', 'gyroscope_y', 'gyroscope_z']:
        fig2.add_trace(go.Scatter(x=data['seconds_elapsed'], y=data[ax], name=ax))
    fig2.show()
    fig3 = go.Figure()
    for ax in ['magnetometer_x', 'magnetometer_y', 'magnetometer_z']:
        fig3.add_trace(go.Scatter(x=data['seconds_elapsed'], y=data[ax], name=ax))
    fig3.show()

In [5]:
# Home
plot_movement(home_class)

In [6]:
plot_movement(garage_class)

In [7]:
plot_movement(roam_class)

In [8]:
def plot_map(data):
    coordinates = [(row.location_latitude, row.location_longitude) for _, row in data.iterrows()]
    path = folium.Map(location=[data.location_latitude.mean(), data.location_longitude.mean()], zoom_start=15)
    folium.PolyLine(coordinates, color='red', weight=3.5).add_to(path)
    display(path)

In [9]:
plot_map(home_class)

In [10]:
plot_map(garage_class)

In [11]:
plot_map(roam_class)

### Accelerometer data

In [15]:
def compute_AccelMagnitude(data):
    data['accel_magnitude'] = np.sqrt((data['accelerometer_x']**2+data['accelerometer_y']**2+data['accelerometer_z']**2))
    jerk_df = np.diff(data[['accelerometer_x', 'accelerometer_y', 'accelerometer_z']], axis=0) * 1
    return data, jerk_df

In [16]:
roam_class, roamJerk_df = compute_AccelMagnitude(roam_class)

In [18]:
home_class, homeJerk_df = compute_AccelMagnitude(home_class)

In [19]:
garage_class, garageJerk_df = compute_AccelMagnitude(garage_class)

In [20]:
fig = go.Figure()
for ax in ['accel_magnitude']:
    fig.add_trace(go.Scatter(x=home_class['seconds_elapsed'], y=home_class[ax], name='Home'))
    fig.add_trace(go.Scatter(x=garage_class['seconds_elapsed'], y= garage_class[ax], name='Garage'))
    fig.add_trace(go.Scatter(x=roam_class['seconds_elapsed'], y=roam_class[ax], name='Roam'))
fig.show()

In [28]:
homeJerk_df = pd.DataFrame(homeJerk_df, columns=['x', 'y', 'z'])
garageJerk_df = pd.DataFrame(garageJerk_df, columns=['x', 'y', 'z'])
roamJerk_df = pd.DataFrame(roamJerk_df, columns=['x', 'y', 'z'])

#homeJerk_df

,x,y,z
0,-0.000238,0.002816,-0.010133
1,-0.001611,0.000531,0.012099
2,0.003916,0.003174,0.013445
3,-0.001116,0.002105,0.015506
4,0.001955,-0.003341,-0.016151
...,...,...,...
541,0.001756,-0.000365,0.009189
542,0.001820,0.001289,0.015524
543,0.000760,0.030140,-0.023082
544,-0.000194,-0.083863,-0.003840


In [29]:
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=homeJerk_df.index, y=homeJerk_df['x'], name='accel_x'))
fig4.add_trace(go.Scatter(x=homeJerk_df.index, y=homeJerk_df['y'], name='accel_y'))
fig4.add_trace(go.Scatter(x=homeJerk_df.index, y=homeJerk_df['z'], name='accel_z'))
fig4.show()

In [30]:
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=garageJerk_df.index, y=garageJerk_df['x'], name='accel_x'))
fig5.add_trace(go.Scatter(x=garageJerk_df.index, y=garageJerk_df['y'], name='accel_y'))
fig5.add_trace(go.Scatter(x=garageJerk_df.index, y=garageJerk_df['z'], name='accel_z'))
fig5.show()

In [31]:
fig6 = go.Figure()
fig6.add_trace(go.Scatter(x=roamJerk_df.index, y=roamJerk_df['x'], name='accel_x'))
fig6.add_trace(go.Scatter(x=roamJerk_df.index, y=roamJerk_df['y'], name='accel_y'))
fig6.add_trace(go.Scatter(x=roamJerk_df.index, y=roamJerk_df['z'], name='accel_z'))
fig6.show()

### Gyroscope data